### Visualizing the learning of a neural network

#### prerequisite 
* Download [tiny-imagenet-challange](https://www.kaggle.com/competitions/tiny-imagenet/overview) data
* Install dependencies 

In [2]:
#imports
import torch
import torchvision 
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot  as plt
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
import time
from tqdm import tqdm
import os

In [3]:
vgg_16 = torchvision.models.vgg16()
vgg_16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
# ! pip install torchinfo

from torchinfo import summary
summary(vgg_16, input_size=(1,3,64,64))

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 2, 2]            --
│    └─Conv2d: 2-1                       [1, 64, 64, 64]           1,792
│    └─ReLU: 2-2                         [1, 64, 64, 64]           --
│    └─Conv2d: 2-3                       [1, 64, 64, 64]           36,928
│    └─ReLU: 2-4                         [1, 64, 64, 64]           --
│    └─MaxPool2d: 2-5                    [1, 64, 32, 32]           --
│    └─Conv2d: 2-6                       [1, 128, 32, 32]          73,856
│    └─ReLU: 2-7                         [1, 128, 32, 32]          --
│    └─Conv2d: 2-8                       [1, 128, 32, 32]          147,584
│    └─ReLU: 2-9                         [1, 128, 32, 32]          --
│    └─MaxPool2d: 2-10                   [1, 128, 16, 16]          --
│    └─Conv2d: 2-11                      [1, 256, 16, 16]          29

In [ ]:
train_ds_path = "../input/tiny-imagenet-challenge/TinyImageNet/train"
torch.manual_seed(0)
tiny_image_net_dataset = datasets.ImageFolder(train_ds_path, 
                                              transform = transforms.Compose([transforms.ToTensor(),
                                              transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
                                                                             ]))
tiny_image_net_dataloader = DataLoader(tiny_image_net_dataset, batch_size=512, shuffle=True)



In [ ]:
image, label =  next(iter(tiny_image_net_dataloader))
i = 0
plt.title(f"{label[i]}")
plt.imshow(image[i].permute(1,2,0))
  

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
vgg_16 = vgg_16.to(device)

crt_CEL = nn.CrossEntropyLoss()

optm_vgg_16 = optim.SGD(vgg_16.parameters(), lr=0.001, momentum=0.9)

sch_cycl_lr = lr_scheduler.CyclicLR(optm_vgg_16,base_lr=0.001,max_lr=0.01, gamma=0.1)

In [ ]:
def train(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_acc = 0.0
    model_dir = 'model/vgg16/'
#     os.makedirs(model_dir)
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
      
        running_loss = 0.0
        running_corrects = 0

        batch = 0 
        for i ,(inputs, labels) in enumerate (tqdm(tiny_image_net_dataloader)):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            scheduler.step()



        epoch_loss = running_loss / len(tiny_image_net_dataset)
        epoch_acc = running_corrects.double() / len(tiny_image_net_dataset)



        weights = dict()    
        for param in parameters :
                weights[param] = model.state_dict()[param]  

        torch.save({"Model_params":weights ,
                   "EPOCH":epoch,
                   "Loss":epoch_loss},f"{model_dir}/vgg16_{epoch}.pt")
        print(f' Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


       
           
        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    return model

In [ ]:
train(model=vgg_16, criterion=crt_CEL, optimizer=optm_vgg_16, scheduler=sch_cycl_lr, num_epochs=64)

In [ ]:
parameters = ["features.0.weight","features.2.weight","features.5.weight","features.7.weight","features.10.weight",             
                "features.12.weight","features.14.weight","features.17.weight","features.19.weight","features.21.weight",
                "features.24.weight","features.26.weight", "features.28.weight"]

In [ ]:
vgg16_wts = torch.load("model/vgg16/vgg16_0.pt")["Model_params"]
# print(model_params[parameters[-1]].shape)
i = 0
layer_i_sum = torch.sum(vgg16_wts[parameters[i]],dim=1) 
upscale = torch.nn.functional.interpolate(torch.unsqueeze(layer_i_sum, dim=1),
                                          size = (64,64),mode='nearest').cpu()


In [ ]:
from pathlib import Path
for wts_file in sorted(os.listdir("model/vgg16"), key = lambda x : int(x[6:-3])):
    vgg16_wts = torch.load(f"model/vgg16/{wts_file}")["Model_params"]
#     print(wts_file)
    for parameter in parameters :
        layer_i_sum = torch.sum(vgg16_wts[parameter], dim=1) 
        upscale = torch.nn.functional.interpolate(torch.unsqueeze((layer_i_sum ), dim=1),
                                                  size=(128,128), mode='nearest').cpu()
        for i in range(len(upscale)) :
            p = f"model/vgg16_imgs/{parameter}/{i}/"
            Path(p).mkdir(parents=True, exist_ok=True )
            cv2.imwrite(f"{p}/{str(wts_file[6:-3]).zfill(2)}.png",
                        ((upscale[i]+1)*127).permute(1,2,0).numpy())
            

In [ ]:
import glob
from PIL import Image
def make_gif(frame_folder, name):
    frames = [Image.open(image) for image in glob.glob(f"{frame_folder}/*")]
    frame_one = frames[0]
    frame_one.save(name, format="GIF", append_images=frames,
               save_all=True, duration=512, loop=1)
    
# for ft in parameters :
#     for     
#         make_gif(f"model/vgg16_imgs/{ft}/{layer}/", f"model/gif/{ft.replace('.','')}_{layer}.gif")    
           
    
for ft in os.listdir("model/vgg16_imgs/")   :
    for layer in os.listdir(f"model/vgg16_imgs/{ft}") :
            make_gif(f"model/vgg16_imgs/{ft}/{layer}/", f"model/gif/{ft.replace('.','')}_{layer}.gif")    
          